In [16]:
import os
import cv2 as cv
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import tensorflow as tf


warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [5]:
# Get Training Data (Original+Mask) DONT RUN IN THIS CODE 
# Dataset 1: HGR
TRAIN_PATH1 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset1_HGR/original_images/']
MASK_PATH1 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset1_HGR/skin_masks/']
train_ids1 = next(os.walk(TRAIN_PATH1[0]))[2]
mask_ids1 = next(os.walk(MASK_PATH1[0]))[2]
train_ids1.sort()
mask_ids1.sort()
TRAIN_PATH1 = TRAIN_PATH1*len(train_ids1)
MASK_PATH1 = MASK_PATH1*len(train_ids1)

# Dataset 4: Pratheepan
TRAIN_PATH2 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset4_Pratheepan/original_images/']
MASK_PATH2 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset4_Pratheepan/skin_masks/']
train_ids2 = next(os.walk(TRAIN_PATH2[0]))[2]
mask_ids2 = next(os.walk(MASK_PATH2[0]))[2]
train_ids2.sort()
mask_ids2.sort()
TRAIN_PATH2 = TRAIN_PATH2*len(train_ids2)
MASK_PATH2 = MASK_PATH2*len(train_ids2)

# Dataset 5: VDM
TRAIN_PATH3 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset5_VDM/original_images/']
MASK_PATH3 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset5_VDM/skin_masks/']
train_id3 = next(os.walk(TRAIN_PATH3[0]))[2]
mask_id3 = next(os.walk(MASK_PATH3[0]))[2]
train_id3.sort()
mask_id3.sort()
train_ids3 = train_id3[1:]
mask_ids3 = mask_id3[1:]
TRAIN_PATH3 = TRAIN_PATH3*len(train_ids3)
MASK_PATH3 = MASK_PATH3*len(train_ids3)

# Dataset 6: SFA
TRAIN_PATH4 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset6_SFA/original_images/']
MASK_PATH4 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset6_SFA/skin_masks/']
train_ids4 = next(os.walk(TRAIN_PATH4[0]))[2]
mask_ids4 = next(os.walk(MASK_PATH4[0]))[2]
train_ids4.sort()
mask_ids4.sort()
TRAIN_PATH4 = TRAIN_PATH4*len(train_ids4)
MASK_PATH4 = MASK_PATH4*len(train_ids4)

# Dataset 2: TDSD
TRAIN_PATH5 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset2_TDSD/original_images/']
MASK_PATH5 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset2_TDSD/skin_masks/']
train_ids5 = next(os.walk(TRAIN_PATH5[0]))[2]
mask_ids5 = next(os.walk(MASK_PATH5[0]))[2]
train_ids5.sort()
mask_ids5.sort()
TRAIN_PATH5 = TRAIN_PATH5*len(train_ids5)
MASK_PATH5 = MASK_PATH5*len(train_ids5)

# Dataset 3: Schmugge
TRAIN_PATH6 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset3_Schmugge/original_images/']
MASK_PATH6 = ['/home/lalzogbi/Documents/Skin_Datasets/Dataset3_Schmugge/skin_masks/']
train_ids6 = next(os.walk(TRAIN_PATH6[0]))[2]
mask_ids6 = next(os.walk(MASK_PATH6[0]))[2]
train_ids6.sort()
mask_ids6.sort()
TRAIN_PATH6 = TRAIN_PATH6*len(train_ids6)
MASK_PATH6 = MASK_PATH6*len(train_ids6)

# Combine everything
TRAIN_PATH = np.concatenate((TRAIN_PATH1,TRAIN_PATH2,TRAIN_PATH3,TRAIN_PATH4,TRAIN_PATH5,TRAIN_PATH6))
MASK_PATH = np.concatenate((MASK_PATH1,MASK_PATH2,MASK_PATH3,MASK_PATH4,MASK_PATH5,MASK_PATH6))
train_ids = np.concatenate((train_ids1,train_ids2,train_ids3,train_ids4,train_ids5,train_ids6))
mask_ids = np.concatenate((mask_ids1,mask_ids2,mask_ids3,mask_ids4,mask_ids5,mask_ids6))

In [7]:
# Get and resize train images and masks DONT RUN IN THIS CODE
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH[n] + id_
    img = imread(path)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img

for n, id_ in tqdm(enumerate(mask_ids), total=len(mask_ids)):
    path = MASK_PATH[n] + id_
    img = imread(path)
    if n in range(899,977):
        img = img[:,:,1]
    img = np.expand_dims(resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
    Y_train[n] = img

Getting and resizing train images and masks ... 


100%|██████████| 3780/3780 [00:07<00:00, 485.84it/s]


In [8]:
# Extract Features and Corresponding Labels
dat = X_train[0]
dat = dat.reshape(16384,3)
hsv = cv.cvtColor(X_train[0], cv.COLOR_RGB2HSV)
hsv = hsv.reshape(16384,3)
lab = cv.cvtColor(X_train[0], cv.COLOR_RGB2Lab)
lab = lab.reshape(16384,3)
L = Y_train[0]
L = L.reshape(16384,1)
dat = np.concatenate((dat,hsv,lab,L),axis=1)

for i in range(len(train_ids)-1):
    i = i+1
    a = X_train[i]
    b = a.reshape(16384,3)
    hsv = cv.cvtColor(X_train[i], cv.COLOR_RGB2HSV)
    hsv = hsv.reshape(16384,3)
    lab = cv.cvtColor(X_train[0], cv.COLOR_RGB2Lab)
    lab = lab.reshape(16384,3)
    L = Y_train[i]
    L = L.reshape(16384,1)
    dat_temp = np.concatenate((b,hsv,lab,L),axis=1)
    dat = np.concatenate((dat,dat_temp))

In [9]:
np.random.shuffle(dat)
feat = dat[0:300000,0:9]
labels = dat[0:300000,9]

In [ ]:
# Build the network
model = Sequential()
model.add(Dense(9, input_dim=9, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2, input_shape=(32,)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2, input_shape=(64,)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2, input_shape=(128,)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2, input_shape=(64,)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2, input_shape=(32,)))
model.add(Dense(1, activation='sigmoid'))

checkpointer = ModelCheckpoint('model-features-5Feb2019.h5', verbose=1, save_best_only=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(feat, labels,validation_split=0.3, epochs=50, batch_size=10, shuffle=True, 
          callbacks=[checkpointer])

Train on 210000 samples, validate on 90000 samples
Epoch 1/50
210000/210000 [==============================] - 80s 382us/step - loss: 0.3241 - acc: 0.8703 - val_loss: 0.3175 - val_acc: 0.8665

Epoch 00001: val_loss improved from inf to 0.31750, saving model to model-features-5Feb2019.h5
Epoch 2/50
210000/210000 [==============================] - 80s 380us/step - loss: 0.2729 - acc: 0.8952 - val_loss: 0.2854 - val_acc: 0.8885

Epoch 00002: val_loss improved from 0.31750 to 0.28542, saving model to model-features-5Feb2019.h5
Epoch 3/50
210000/210000 [==============================] - 80s 380us/step - loss: 0.2686 - acc: 0.8967 - val_loss: 0.2561 - val_acc: 0.9046

Epoch 00003: val_loss improved from 0.28542 to 0.25611, saving model to model-features-5Feb2019.h5
Epoch 4/50
210000/210000 [==============================] - 80s 379us/step - loss: 0.2666 - acc: 0.8979 - val_loss: 0.2456 - val_acc: 0.9038

Epoch 00004: val_loss improved from 0.25611 to 0.24558, saving model to model-features-5


Epoch 00041: val_loss did not improve from 0.24198
Epoch 42/50
210000/210000 [==============================] - 80s 380us/step - loss: 1.6962 - acc: 0.8354 - val_loss: 3.8100 - val_acc: 0.7636

Epoch 00042: val_loss did not improve from 0.24198
Epoch 43/50
210000/210000 [==============================] - 80s 379us/step - loss: 2.2319 - acc: 0.8126 - val_loss: 0.3244 - val_acc: 0.8785

Epoch 00043: val_loss did not improve from 0.24198
Epoch 44/50
210000/210000 [==============================] - 79s 378us/step - loss: 0.3330 - acc: 0.8842 - val_loss: 0.3239 - val_acc: 0.8997

Epoch 00044: val_loss did not improve from 0.24198
Epoch 45/50
210000/210000 [==============================] - 79s 378us/step - loss: 0.3347 - acc: 0.8834 - val_loss: 0.2702 - val_acc: 0.8952

Epoch 00045: val_loss did not improve from 0.24198
Epoch 46/50
210000/210000 [==============================] - 80s 380us/step - loss: 0.3096 - acc: 0.8838 - val_loss: 0.2763 - val_acc: 0.8981

Epoch 00046: val_loss did not

In [ ]:
# Evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))